# Data Cleaning Process




# The cell below cleans the directory
so that when the data base is updated, first the directory is cleaned and then new files are added to it.

In [1]:
#clean the directory
import os

def delete_files_in_directory(directory):
    # Check if the directory exists
    if not os.path.exists(directory):
        print("Directory does not exist:", directory)
        return

  
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # This will delete the file or symlink
            elif os.path.isdir(file_path):
                # If you want to delete directories, uncomment the following line
                # shutil.rmtree(file_path)
                pass
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))


delete_files_in_directory('/Users/karankooshavar/Downloads/archive/alls')
print('fatto')

fatto


# The four cells below are just to convert the files from .tsv to .csv
Just because working with .csv is easier and more familiar to most users.

In [2]:
import pandas as pd

# Load the large dataset specifying the tab separator
df = pd.read_csv('/Users/karankooshavar/Downloads/archive/title.basics.tsv/data.tsv', sep='\t')

# Save the smaller sample dataset as CSV
df.to_csv('/Users/karankooshavar/Downloads/archive/alls/all_titles.csv', index=False)
print('fatto')


/var/folders/wc/n_yxy54n7g94rjjzy4vskbkr0000gn/T/ipykernel_1720/2945633562.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/karankooshavar/Downloads/archive/title.basics.tsv/data.tsv', sep='\t')


fatto


In [3]:
#import pandas as pd

# Load the large dataset specifying the tab separator
df = pd.read_csv('/Users/karankooshavar/Downloads/archive/title.ratings.tsv/data.tsv', sep='\t')

# Save the smaller sample dataset as CSV
df.to_csv('/Users/karankooshavar/Downloads/archive/alls/all_ratings.csv', index=False)
print('fatto')


fatto


In [4]:
#import pandas as pd

# Load the large dataset specifying the tab separator
df = pd.read_csv('/Users/karankooshavar/Downloads/archive/name.basics.tsv/data.tsv', sep='\t')

# Save the smaller sample dataset as CSV
df.to_csv('/Users/karankooshavar/Downloads/archive/alls/all_people.csv', index=False)
print('fatto')


fatto


In [5]:
#import pandas as pd

# Load the large dataset specifying the tab separator
df = pd.read_csv('/Users/karankooshavar/Downloads/archive/title.principals.tsv/data.tsv', sep='\t')

# Save the smaller sample dataset as CSV
df.to_csv('/Users/karankooshavar/Downloads/archive/alls/all_roles.csv', index=False)
print('fatto')


fatto


# Cleaning the movie titles
The cell uses 2 different tables to determin which movies are rated more than 6.5 and are rated by more that 100000 users. This part is to delete the less known movies from the data set, so the ordinary user is familiar with most of the movies that are questioned in the quiz.

In [6]:
#import pandas as pd

# Paths to the CSV files
ratings_path = '/Users/karankooshavar/Downloads/archive/alls/all_ratings.csv'
titles_path = '/Users/karankooshavar/Downloads/archive/alls/all_titles.csv'

# Load the data from both files into pandas dataframes
ratings_df = pd.read_csv(ratings_path)
titles_df = pd.read_csv(titles_path)

# Filter the ratings dataframe based on the given criteria
filtered_ratings_df = ratings_df[(ratings_df['numVotes'] > 100000) & (ratings_df['averageRating'] > 6.5)]

# Merge the filtered ratings dataframe with the titles dataframe on 'tconst'
merged_df = pd.merge(filtered_ratings_df, titles_df, on='tconst', how='inner')

# Path to the output CSV file
output_path = 'filtered_titles.csv'

# Save the merged dataframe to a CSV file
merged_df.to_csv('/Users/karankooshavar/Downloads/archive/alls/all_filtered_titles.csv', index=False)

print('fatto')


/var/folders/wc/n_yxy54n7g94rjjzy4vskbkr0000gn/T/ipykernel_1720/475274057.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles_df = pd.read_csv(titles_path)


fatto


This cell drops the columns in on the table that are not needed for our quiz.

In [7]:
#import pandas as pd

# Path to the CSV file
input_path = '/Users/karankooshavar/Downloads/archive/alls/all_filtered_titles.csv'

# Load the data into a pandas dataframe
df = pd.read_csv(input_path)

# Drop the specified columns
df = df.drop(columns=['originalTitle', 'isAdult', 'runtimeMinutes'])

# Path to the output CSV file
output_path = 'cleaned_titles.csv'

# Save the dataframe to a new CSV file
df.to_csv('/Users/karankooshavar/Downloads/archive/alls/all_cleaned_titles.csv', index=False)

print('fatto')


fatto


# Cleaning the roles table
In the roles table we have Person-ID and Movie-ID of everyone ever in the movies. This cell only keeps the roles that the corresponding Movie-ID is in our previously cleaned and filtered movies dataset.

In [8]:
#import pandas as pd

# Load the data from both files into pandas dataframes
cleaned_titles_path = '/Users/karankooshavar/Downloads/archive/alls/all_cleaned_titles.csv'
roles_path = '/Users/karankooshavar/Downloads/archive/alls/all_roles.csv'

cleaned_titles_df = pd.read_csv(cleaned_titles_path)
roles_df = pd.read_csv(roles_path)

# Filter roles_df to only keep rows with tconst present in cleaned_titles_df
filtered_roles_df = roles_df[roles_df['tconst'].isin(cleaned_titles_df['tconst'])]

# Save the filtered dataframe to a CSV file
output_roles_path = 'filtered_roles.csv'
filtered_roles_df.to_csv('/Users/karankooshavar/Downloads/archive/alls/all_cleaned_roles.csv', index=False)

print('fatto')


fatto


# Adding difficulty to each movie
By using the K-means clustring, the cell below uses the YEAR, NUMBER OF VOTES and Average rating to cluster the movie titles to 3 groups. Then assigning them a difficulty score where 3 is the  most difficult and 1 is the easiest.

In [13]:
#import pandas as pd
from sklearn.cluster import KMeans


file_path = '/Users/karankooshavar/Downloads/archive/alls/all_cleaned_titles.csv'
data = pd.read_csv(file_path)


# Selecting the relevant columns for clustering
X = data[['averageRating','numVotes', 'startYear']]

# KMeans with 3 clusters
kmeans = KMeans(n_clusters=3, random_state=0).fit(X)

# Assigning the cluster labels to a new column in the dataframe
data['difficulty'] = kmeans.labels_ + 1  # Adding 1 to make range 1-3 instead of 0-2

# Swap 1 with 3 and 3 with 1
data['difficulty'] = data['difficulty'].replace({1: 3, 3: 1})


# Saving the modified dataframe to a new CSV file
data.to_csv('/Users/karankooshavar/Downloads/archive/alls/all_with_difficulty_titles.csv', index=False)

print('fatto')

fatto


Because the roles table doesn't include the name of the people amd just the Person-ID, this cell tries to merge two tables and put the name of the person according to their ID in the same table.

In [10]:
#import pandas as pd

# Load the data from both files into pandas dataframes
cleaned_roles_path = '/Users/karankooshavar/Downloads/archive/alls/all_cleaned_roles.csv'
allpeople_path = '/Users/karankooshavar/Downloads/archive/alls/all_people.csv'

cleaned_roles_df = pd.read_csv(cleaned_roles_path)
allpeople_df = pd.read_csv(allpeople_path)

# Filter roles_df to only keep rows with tconst present in cleaned_titles_df
filtered_roles_df = allpeople_df[allpeople_df['nconst'].isin(cleaned_roles_df['nconst'])]

# Save the filtered dataframe to a CSV file
output_roles_path = 'people_cleaned_roles.csv'
filtered_roles_df.to_csv('/Users/karankooshavar/Downloads/archive/alls/people_cleaned_roles.csv', index=False)

print('fatto')

fatto


Finally the data from all the files above are combined to have a table with all the data we want for the quiz.

In [11]:
#import pandas as pd

# Load the data from both files into pandas dataframes
cleaned_titles_path = '/Users/karankooshavar/Downloads/archive/alls/all_with_difficulty_titles.csv'
roles_path = '/Users/karankooshavar/Downloads/archive/alls/all_cleaned_roles.csv'
people_cleaned_path = "/Users/karankooshavar/Downloads/archive/alls/people_cleaned_roles.csv"


cleaned_diffuculty_titles_df = pd.read_csv(cleaned_titles_path)
roles_cleaned_df = pd.read_csv(roles_path)
people_cleaned_roles_df = pd.read_csv(people_cleaned_path)

# Merge the filtered ratings dataframe with the titles dataframe on 'tconst'
merged_quiz_df = pd.merge(roles_cleaned_df[['tconst', 'nconst', 'category']],
                     cleaned_diffuculty_titles_df[['tconst', 'titleType', 'primaryTitle', 'startYear','difficulty']],
                     on='tconst', how='inner')
merged_quiz2_df = pd.merge(merged_quiz_df,
                     people_cleaned_roles_df[['nconst', 'primaryName']],
                     on='nconst', how='inner')

# Save the filtered dataframe to a CSV file
output_roles_path = 'quizable_roles.csv'
merged_quiz2_df.to_csv('/Users/karankooshavar/Downloads/archive/alls/all_qiuzable_roles.csv', index=False)

print('fatto')

fatto
